In [1]:
import tqdm
import random
import itertools
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import cm

import xgboost as xgb

plt.style.use("jupyter")

In [2]:
dm = pd.read_csv("../data/iza_dm.csv", index_col=0)
synth = pd.read_csv("../data/synthesis_fraction.csv", index_col=0)
synth = synth.loc[dm.index]

## Training classifiers

In [3]:
BEST_HYPERPARAMS = {
    "colsample_bytree": 0.5,
    "learning_rate": 0.1,
    "max_depth": 6,
    "min_child_weight": 1,
    "n_estimators": 200,
    "subsample": 0.5
}

def get_classifier(**params):
    return xgb.XGBClassifier(
        objective="binary:logistic",
        **BEST_HYPERPARAMS,
        **params,
    )

In [4]:
clf = get_classifier()

for 